In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

TSLA= yf.download("TSLA", start="2020-01-01",end="2025-08-29",multi_level_index=False)
APPLE= yf.download("AAPL", start="2020-01-01",end="2025-08-29",multi_level_index=False)
XOM= yf.download("XOM", start="2020-01-01",end="2025-08-29",multi_level_index=False)
KO= yf.download("KO", start="2020-01-01",end="2025-08-29",multi_level_index=False)

/tmp/ipython-input-3368548178.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  TSLA= yf.download("TSLA", start="2020-01-01",end="2025-08-29",multi_level_index=False)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3368548178.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  APPLE= yf.download("AAPL", start="2020-01-01",end="2025-08-29",multi_level_index=False)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3368548178.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  XOM= yf.download("XOM", start="2020-01-01",end="2025-08-29",multi_level_index=False)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-3368548178.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  KO= yf.download("KO", start="2020-01-01",end="2025-08-29",multi_leve

In [9]:
TSLA["Returns"]= TSLA["Close"].pct_change()
APPLE["Returns"]= APPLE["Close"].pct_change()
XOM["Returns"]= XOM["Close"].pct_change()
KO["Returns"]= KO["Close"].pct_change()

In [10]:
portfolio_returns_df = pd.DataFrame({
    'TSLA': TSLA["Returns"],
    'APPLE': APPLE["Returns"],
    'XOM': XOM["Returns"],
    'KO': KO["Returns"]
}).ffill().dropna()

##Calculation of the VAR and ES of the riskier portfolio
VAR is calculated using the non parametric model, in order to avoid assumptions on the distribution but focusing only on the data themselves.

#Dashboard through Dash

In [6]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.3 MB/s eta 0:00:00


In [21]:
!pip install dash-bootstrap-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 5.4 MB/s eta 0:00:00


In [7]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc

In [31]:
app = Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.layout = dbc.Container([

    # Title
    dbc.Row([
        dbc.Col(html.H1("📊 Portfolio Risk Dashboard",
                        style={'textAlign': 'center'}), width=12)
    ], className="mb-4"),

    # Inputs row with labels
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            ['TSLA', 'APPLE', 'XOM', 'KO'],
            multi=True, id='items',
            value=['TSLA', 'APPLE', 'XOM', 'KO']), width=4),

        dbc.Col([
            dbc.InputGroup([
                dbc.InputGroupText("Holding Period"),
                dbc.Input(id="period", type="number", value=1)
            ], className="mb-2"),

            dbc.InputGroup([
                dbc.InputGroupText("Confidence Level"),
                dbc.Input(id="level", type="number", value=0.95, step=0.01)
            ], className="mb-2"),
        ], width=3),

        dbc.Col([
            dbc.InputGroup([dbc.InputGroupText("Weight A"), dbc.Input(id="weight A", type="number", value=0.25)]),
            dbc.InputGroup([dbc.InputGroupText("Weight B"), dbc.Input(id="weight B", type="number", value=0.25)]),
            dbc.InputGroup([dbc.InputGroupText("Weight C"), dbc.Input(id="weight C", type="number", value=0.25)]),
            dbc.InputGroup([dbc.InputGroupText("Weight D"), dbc.Input(id="weight D", type="number", value=0.25)]),
        ], width=5),
    ], className="mb-4"),

    # Graphs row
    dbc.Row([
        dbc.Col(dcc.Graph(id='controls'), width=6),
        dbc.Col(dcc.Graph(id='controls 2'), width=6),
    ], className="mb-4"),

    # Risk metrics row
    dbc.Row([
        dbc.Col(html.Div(id="risk_metrics",
                         style={'textAlign': 'center', 'fontSize': 20}), width=12)
    ], className="mb-4"),

    # DataTables row with custom styling
    dbc.Row([
        dbc.Col(dash_table.DataTable(id="apple", page_size=10,
                                     style_table={'overflowX': 'auto'},
                                     style_header={'backgroundColor': '#1f2c56','color': 'white','fontWeight': 'bold','textAlign': 'center'},
                                     style_cell={'backgroundColor': '#0e1117','color': 'white','textAlign': 'center','padding': '8px'},
                                     style_data_conditional=[
                                         {'if': {'row_index': 'odd'}, 'backgroundColor': '#1a1d23'},
                                         {'if': {'state': 'active'}, 'backgroundColor': '#2a3f5f','border': '1px solid white'}
                                     ]), width=3),

        dbc.Col(dash_table.DataTable(id="tsla", page_size=10, style_table={'overflowX': 'auto'},
                                     style_header={'backgroundColor': '#1f2c56','color': 'white','fontWeight': 'bold','textAlign': 'center'},
                                     style_cell={'backgroundColor': '#0e1117','color': 'white','textAlign': 'center','padding': '8px'},
                                     style_data_conditional=[
                                         {'if': {'row_index': 'odd'}, 'backgroundColor': '#1a1d23'},
                                         {'if': {'state': 'active'}, 'backgroundColor': '#2a3f5f','border': '1px solid white'}
                                     ]), width=3),

        dbc.Col(dash_table.DataTable(id="xom", page_size=10, style_table={'overflowX': 'auto'},
                                     style_header={'backgroundColor': '#1f2c56','color': 'white','fontWeight': 'bold','textAlign': 'center'},
                                     style_cell={'backgroundColor': '#0e1117','color': 'white','textAlign': 'center','padding': '8px'},
                                     style_data_conditional=[
                                         {'if': {'row_index': 'odd'}, 'backgroundColor': '#1a1d23'},
                                         {'if': {'state': 'active'}, 'backgroundColor': '#2a3f5f','border': '1px solid white'}
                                     ]), width=3),

        dbc.Col(dash_table.DataTable(id="ko", page_size=10, style_table={'overflowX': 'auto'},
                                     style_header={'backgroundColor': '#1f2c56','color': 'white','fontWeight': 'bold','textAlign': 'center'},
                                     style_cell={'backgroundColor': '#0e1117','color': 'white','textAlign': 'center','padding': '8px'},
                                     style_data_conditional=[
                                         {'if': {'row_index': 'odd'}, 'backgroundColor': '#1a1d23'},
                                         {'if': {'state': 'active'}, 'backgroundColor': '#2a3f5f','border': '1px solid white'}
                                     ]), width=3),
    ])
], fluid=True)



def calculate_var(portfolio_ret, weight_A, weight_B, weight_C, weight_D, confidence_level, holding_period):
    """
    Compute Value-at-Risk (VaR) and Expected Shortfall (ES).
    """

    # Value at Risk
    var = np.percentile(portfolio_ret, (1 - confidence_level) * 100) * np.sqrt(holding_period)

    # Expected Shortfall
    worst_returns = portfolio_ret[portfolio_ret < var]
    es = worst_returns.mean() * np.sqrt(holding_period) if not worst_returns.empty else np.nan

    return var, es


@callback(
    Output('apple', 'data'),
    Output('tsla', 'data'),
    Output('xom', 'data'),
    Output('ko', 'data'),
    Output('controls', 'figure'),
    Output('controls 2', 'figure'),
    Output('risk_metrics', 'children'),
    Input('items', 'value'),
    Input('period', 'value'),
    Input('level', 'value'),
    Input('weight A', 'value'),
    Input('weight B', 'value'),
    Input('weight C', 'value'),
    Input('weight D', 'value'),
)
def update_output(col_chosen, holding_period, confidence_level,
                  weight_A, weight_B, weight_C, weight_D):

    filtered_df = portfolio_returns_df[col_chosen]

    # Normalize weights
    weights = np.array([weight_A, weight_B, weight_C, weight_D], dtype=float)
    if weights.sum() == 0:
        weights = np.ones(4) / 4  # fallback equal weights
    else:
        weights = weights / weights.sum()
    weight_A, weight_B, weight_C, weight_D = weights

    # Line chart of chosen assets
    fig = px.line(filtered_df, y=col_chosen)

    # Update tables only for selected assets
    apple_data = filtered_df[['APPLE']].to_dict('records') if 'APPLE' in col_chosen else []
    tsla_data = filtered_df[['TSLA']].to_dict('records') if 'TSLA' in col_chosen else []
    xom_data = filtered_df[['XOM']].to_dict('records') if 'XOM' in col_chosen else []
    ko_data = filtered_df[['KO']].to_dict('records') if 'KO' in col_chosen else []

    # Portfolio returns (using .get to avoid KeyError)
    portfolio_ret = (
        weight_A * filtered_df.get('APPLE', 0) +
        weight_B * filtered_df.get('TSLA', 0) +
        weight_C * filtered_df.get('XOM', 0) +
        weight_D * filtered_df.get('KO', 0)
    )

    # Histogram of portfolio returns
    fig2 = px.histogram(portfolio_ret, nbins=50, color_discrete_sequence=["red"])

    # Risk metrics
    if holding_period and confidence_level and len(col_chosen) > 0:
        var, es = calculate_var(portfolio_ret, weight_A, weight_B, weight_C, weight_D,
                                confidence_level, holding_period)
        metrics_text = (
            f"📉 Value-at-Risk (VaR): {var:.4f} | "
            f"⚠️ Expected Shortfall (ES): {es:.4f} | "
            f"(Weights normalized: {weights.round(2)})"
        )
    else:
        metrics_text = "Please select assets and provide valid inputs."

    fig2.add_vline(x=var, line_color="blue", line_dash="dash",
                   annotation_text=f"VaR {var:.4f}", annotation_position="top left")
    fig2.add_vline(x=es, line_color="green", line_dash="dot",
                   annotation_text=f"ES {es:.4f}", annotation_position="top left")

    return apple_data, tsla_data, xom_data, ko_data, fig, fig2, metrics_text
# Run the app
if __name__ == '__main__':
  app.run(debug=True)

<IPython.core.display.Javascript object>